In [89]:
import pandas as pd
import re
import os
import json
import time
from shutil import copy2, rmtree
from shutil import copyfile
import shutil
from PIL import Image

In [70]:
data = pd.read_json('evolution2.json', orient='index')

len(data)

904037

In [59]:
'''
def get_itemID(index):
    res = index.split('/')
    if res[-1] == 'feedback' or res[-1] == 'return-policy':
        return int(res[-2])
    elif res[-1].find('.') >=0:
        return int(res[-1].split('.')[0])
    else:
        return int(res[-1])

data['itemID'] = map(get_itemID, list(data.index.values))

data.to_json('evolution2.json', orient='index')
'''

In [71]:
root_path = '/media/intel/m2/evolution/'

In [83]:
data['image_location'] = root_path + data.date.astype(str) + '/' + data.img

In [84]:
data = data[data.image_location.notnull()].copy()

In [85]:
f_path = lambda x: '/'.join(x.split('/')[:-2])
data['image_location'] = data.image_location.apply(f_path)

### seller identification

In [90]:
data = data[data.seller.notnull()]

In [91]:
target_path = '/media/intel/m2/imgs/Evo'
try:
    rmtree(target_path)
except:
    pass
try:
    os.makedirs(target_path)
except:
    pass

In [92]:
data['seller_name'] = data.seller.apply(lambda x: os.path.join(target_path, x))

In [93]:
img_count = []
for index, row in data.iterrows():
    img_f = row.image_location
    if not os.path.isdir(img_f):
        continue
    for img_index in os.listdir(img_f):
        if img_index == 'none':
            continue
        img_count.append(int(img_index))

In [97]:
from collections import Counter
print Counter(img_count)

Counter({0: 724552, 1: 106846, 2: 55465, 3: 32130, 4: 4976, 5: 1879, 6: 741, 7: 451, 8: 226, 9: 206, 10: 187, 32: 32, 29: 29, 30: 27, 21: 20, 11: 18, 22: 18, 31: 18, 34: 18, 28: 17, 23: 13, 24: 13, 25: 13, 26: 13, 19: 12, 20: 12, 27: 12, 33: 11, 12: 5, 13: 2, 14: 2, 17: 2, 15: 1, 16: 1, 18: 1})


In [101]:
i = 0
for index, row in data.iterrows():
    i += 1
    if i % 3000 == 0:
        print i,
    img_f = row.image_location
    if not os.path.isdir(img_f):
        continue
    for img_index in os.listdir(img_f):
        if img_index == 'none':
            continue
        if os.path.isfile(os.path.join(img_f, img_index, 'large')):
            oriname = 'large'
        elif os.path.isfile(os.path.join(img_f, img_index, 'large.jpg')):
            oriname = 'large.jpg'
        else:
            continue
        img_path = os.path.join(img_f, img_index, oriname)
        try:
            img = Image.open(img_path)
            img.load()
            img.close()
        except IOError:
            img.close()
            continue       
        if not os.path.isdir(row.seller_name):
            os.makedirs(row.seller_name)
        tar_file = os.path.join(row.seller_name, "%d%2.2d.jpg" % (row.itemID, int(img_index)))
        copyfile(img_path, tar_file)

3000 6000 9000 12000 15000 18000 21000 24000 27000 30000 33000 36000 39000 42000 45000 48000 51000 54000 57000 60000 63000 66000 69000 72000 75000 78000 81000 84000 87000 90000 93000 96000 99000 102000 105000 108000 111000 114000 117000 120000 123000 126000 129000 132000 135000 138000 141000 144000 147000 150000 153000 156000 159000 162000 165000 168000 171000 174000 177000 180000 183000 186000 189000 192000 195000 198000 201000 204000 207000 210000 213000 216000 219000 222000 225000 228000 231000 234000 237000 240000 243000 246000 249000 252000 255000 258000 261000 264000 267000 270000 273000 276000 279000 282000 285000 288000 291000 294000 297000 300000 303000 306000 309000 312000 315000 318000 321000 324000 327000 330000 333000 336000 339000 342000 345000 348000 351000 354000 357000 360000 363000 366000 369000 372000 375000 378000 381000 384000 387000 390000 393000 396000 399000 402000 405000 408000 411000 414000 417000 420000 423000 426000 429000 432000 435000 438000 441000 444000 

### dedup

In [63]:
import hashlib
def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(blocksize), b""):
            hash.update(block)
    return hash.hexdigest()

In [64]:
root_dir = '/media/intel/m2/imgEvodedup'
seller_name = [os.path.join(root_dir, x) for x in os.listdir(root_dir)]
for seller_folder in set(seller_name):
    filenames = map(lambda x: os.path.join(seller_folder, x), os.listdir(seller_folder))
    md5value = {}
    for filename in filenames:
        md5curr = md5sum(filename)
        if md5curr in md5value:
            os.remove(filename)
        else:
            md5value[md5curr] = filename

### category prediction

In [12]:
data3 = data.copy()

In [13]:
data3 = data3[data3.cat.notnull()]

In [14]:
data3.cat.unique()

array([u'Services', u'Fraud Related', u'Electronics', u'Digital Goods',
       u'Guides &amp; Tutorials', u'Drugs', u'Counterfeits',
       u'Miscellaneous', u'Erotica', u'Lab Supplies', u'Weapons',
       u'Jewellery', u'Custom Listings', u'Drug Paraphernalia'], dtype=object)

In [15]:
target_path = '/media/intel/m2/imgEvoCat'
try:
    os.mkdir(target_path)
except:
    pass

data3['cat_name'] = data3.cat.apply(lambda x: os.path.join(target_path, re.sub('[\W_]+', '00', x)))

for index, row in data3.iterrows():
    img_f = row.image_location
    if not os.path.isdir(img_f):
        continue
    for img_index in os.listdir(img_f):
        if img_index == 'none':
            continue
        if os.path.isfile(os.path.join(img_f, img_index, 'large')):
            oriname = 'large'
        elif os.path.isfile(os.path.join(img_f, img_index, 'large.jpg')):
            oriname = 'large.jpg'
            '''
            elif os.path.isfile(os.path.join(img_f, img_index, 'tiny')):
                oriname = 'tiny'
            elif os.path.isfile(os.path.join(img_f, img_index, 'tiny.jpg')):
                oriname = 'tiny.jpg'
            '''
        else:
            continue
        img_path = os.path.join(img_f, img_index, oriname)
        try:
            img = Image.open(img_path)
            img.load()
            img.close()
        except IOError:
            img.close()
            continue        
        if not os.path.isdir(row.cat_name):
            os.makedirs(row.cat_name)
        tar_file = os.path.join(row.cat_name, img_f.split('/')[-1] + 'xxxx' + img_index)
        i = 0
        while os.path.isfile("%sxxxx%d.jpg" % (tar_file, i)):
            i += 1
        copyfile(img_path, "%sxxxx%d.jpg" % (tar_file, i))

### dedup

In [16]:
import hashlib
def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(blocksize), b""):
            hash.update(block)
    return hash.hexdigest()

In [17]:
root_dir = '/media/intel/m2/imgEvoCat'
seller_name = [os.path.join(root_dir, x) for x in os.listdir(root_dir)]
for seller_folder in set(seller_name):
    filenames = map(lambda x: os.path.join(seller_folder, x), os.listdir(seller_folder))
    md5value = {}
    for filename in filenames:
        md5curr = md5sum(filename)
        if md5curr in md5value:
            os.remove(filename)
        else:
            md5value[md5curr] = filename